# 簡易 Excel 檔案分享工具

這個筆記本提供簡單的方法來分享 Excel 檔案，不需要 Azure AD 設定。

## 可用選項：
1. 使用 Outlook 自動發送 email（如果已安裝）
2. 生成 SharePoint/OneDrive 分享連結（手動方式）
3. 使用 SMTP 發送 email（如果有 email 伺服器設定）

## 選項 1: 使用 Outlook（Windows）

In [ ]:
import os
import sys
import platform

def share_via_outlook(file_path, recipient_email, subject="分享 Excel 檔案"):
    """使用 Outlook 發送檔案（僅限 Windows）"""
    if platform.system() != 'Windows':
        print("❌ 此方法僅適用於 Windows 系統")
        return False
    
    try:
        import win32com.client
        
        outlook = win32com.client.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        
        mail.To = recipient_email
        mail.Subject = subject
        mail.Body = f"請查收附件中的 Excel 檔案。\n\n檔案名稱：{os.path.basename(file_path)}"
        
        # 附加檔案
        mail.Attachments.Add(file_path)
        
        # 顯示郵件（使用者需要手動點擊發送）
        mail.Display()
        
        print(f"✅ Outlook 郵件已建立，請點擊發送")
        return True
        
    except Exception as e:
        print(f"❌ 無法使用 Outlook: {str(e)}")
        print("請確認：")
        print("1. 已安裝 Outlook")
        print("2. 已安裝 pywin32: pip install pywin32")
        return False

# 使用範例
# share_via_outlook("path/to/file.xlsx", "recipient@email.com")

## 選項 2: 生成 SharePoint/OneDrive 分享指令

In [ ]:
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets

def create_sharing_instructions(excel_file, reviewer_column='Reviewer'):
    """建立手動分享的指令"""
    try:
        # 讀取 Excel 檔案
        df = pd.read_excel(excel_file)
        
        if reviewer_column in df.columns:
            reviewers = df[reviewer_column].dropna().unique()
            
            print(f"找到 {len(reviewers)} 位審查者\n")
            print("=" * 50)
            print("📋 手動分享步驟：\n")
            
            print("1. 將檔案上傳到 SharePoint/OneDrive")
            print("2. 對每個審查者資料夾：")
            print("   - 右鍵點擊資料夾")
            print("   - 選擇『分享』")
            print("   - 輸入對應的 email 地址")
            print("   - 設定權限（檢視或編輯）")
            print("   - 點擊『傳送』\n")
            
            print("=" * 50)
            print("\n📧 審查者清單（請手動輸入 email）：\n")
            
            # 建立一個簡單的表格顯示
            for i, reviewer in enumerate(reviewers, 1):
                print(f"{i}. {reviewer}: ___________________________@company.com")
            
            # 建立可複製的清單
            print("\n📄 可複製的審查者清單：")
            print("-" * 30)
            for reviewer in reviewers:
                print(reviewer)
                
        else:
            print(f"❌ 找不到 '{reviewer_column}' 欄位")
            
    except Exception as e:
        print(f"❌ 錯誤: {str(e)}")

# 使用介面
excel_input = widgets.Text(
    placeholder='輸入 Excel 檔案路徑',
    description='Excel 檔案:',
    layout=widgets.Layout(width='500px')
)

generate_btn = widgets.Button(
    description='生成分享指令',
    layout=widgets.Layout(width='150px')
)

output = widgets.Output()

def on_generate(b):
    with output:
        output.clear_output()
        if excel_input.value:
            create_sharing_instructions(excel_input.value)
        else:
            print("請輸入 Excel 檔案路徑")

generate_btn.on_click(on_generate)

display(excel_input)
display(generate_btn)
display(output)

## 選項 3: 使用系統預設 Email 客戶端

In [ ]:
import webbrowser
import urllib.parse

def create_email_link(recipient, subject="Excel 檔案分享", body="請查收附件"):
    """建立 mailto 連結，開啟預設郵件客戶端"""
    
    # 編碼郵件內容
    subject_encoded = urllib.parse.quote(subject)
    body_encoded = urllib.parse.quote(body)
    
    # 建立 mailto 連結
    mailto_link = f"mailto:{recipient}?subject={subject_encoded}&body={body_encoded}"
    
    print(f"📧 正在開啟郵件客戶端...")
    print(f"收件人: {recipient}")
    print(f"\n注意：您需要手動附加檔案")
    
    # 開啟預設郵件客戶端
    webbrowser.open(mailto_link)
    
    return mailto_link

# 建立互動介面
email_input = widgets.Text(
    placeholder='recipient@company.com',
    description='收件人 Email:',
    layout=widgets.Layout(width='400px')
)

subject_input = widgets.Text(
    value='Excel 檔案分享',
    description='主旨:',
    layout=widgets.Layout(width='400px')
)

send_btn = widgets.Button(
    description='開啟郵件客戶端',
    layout=widgets.Layout(width='150px')
)

def on_send(b):
    if email_input.value:
        create_email_link(email_input.value, subject_input.value)
    else:
        print("請輸入收件人 email")

send_btn.on_click(on_send)

display(email_input)
display(subject_input)
display(send_btn)

## 選項 4: 批次處理多個收件人

In [ ]:
def batch_create_emails(reviewer_emails):
    """為多個審查者建立分享指令"""
    print("📋 批次分享指令\n")
    print("=" * 50)
    
    for reviewer, email in reviewer_emails.items():
        print(f"\n👤 {reviewer}")
        print(f"📧 Email: {email}")
        print(f"📁 資料夾: {reviewer}/")
        print(f"📎 檔案: {reviewer} - data.xlsx")
        
        # 建立 mailto 連結
        subject = f"您的審查資料 - {reviewer}"
        body = f"您好 {reviewer}，\n\n請查收附件中的審查資料。\n\n謝謝！"
        
        mailto = f"mailto:{email}?subject={urllib.parse.quote(subject)}&body={urllib.parse.quote(body)}"
        print(f"🔗 Email 連結: {mailto[:50]}...")
        print("-" * 50)
    
    print("\n💡 提示：")
    print("1. 點擊上方連結會開啟郵件客戶端")
    print("2. 手動附加對應的 Excel 檔案")
    print("3. 或使用 SharePoint 的『分享』功能")

# 範例使用
sample_data = {
    "張三": "zhangsan@company.com",
    "李四": "lisi@company.com",
    "王五": "wangwu@company.com"
}

# batch_create_emails(sample_data)

## 選項 5: 生成 PowerShell 腳本（簡化版）

In [ ]:
def create_simple_powershell_script(folder_path, reviewer_emails):
    """生成簡單的 PowerShell 腳本來開啟 Outlook 郵件"""
    
    script_content = '''# PowerShell 腳本 - 使用 Outlook 分享檔案
$outlook = New-Object -ComObject Outlook.Application

'''
    
    for reviewer, email in reviewer_emails.items():
        file_path = f"{folder_path}\\{reviewer}\\*.xlsx"
        
        script_content += f'''
# 為 {reviewer} 建立郵件
$mail = $outlook.CreateItem(0)
$mail.To = "{email}"
$mail.Subject = "您的審查資料 - {reviewer}"
$mail.Body = "請查收附件中的審查資料。"

# 附加檔案
$files = Get-ChildItem "{file_path}"
foreach ($file in $files) {{
    $mail.Attachments.Add($file.FullName)
}}

$mail.Display()  # 顯示郵件視窗
# $mail.Send()   # 自動發送（取消註解以啟用）

Write-Host "已建立給 {reviewer} 的郵件"
'''
    
    # 儲存腳本
    script_path = "share_files_simple.ps1"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ 已生成 PowerShell 腳本: {script_path}")
    print("\n執行方式：")
    print("1. 開啟 PowerShell")
    print(f"2. 執行: .\\{script_path}")
    print("3. 檢查並發送每封郵件")
    
    return script_path

# 使用範例
# create_simple_powershell_script("C:\\Excel_Output", sample_data)

## 最簡單的解決方案

如果您已經有 SharePoint 存取權限，最簡單的方法是：

### 方法 1: 使用 SharePoint 網頁介面
1. 將資料夾上傳到 SharePoint
2. 對每個資料夾點擊「分享」
3. 輸入收件人 email
4. 選擇權限等級
5. 發送

### 方法 2: 使用 OneDrive 同步
1. 如果資料夾在 OneDrive 同步資料夾中
2. 右鍵點擊資料夾 → 分享
3. 輸入 email 地址

### 方法 3: 使用 Teams
1. 在 Teams 中建立頻道
2. 上傳檔案到頻道
3. 使用 @mention 通知相關人員